In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fintro_prompt_design.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Karl Weinmeister](https://github.com/kweinmeister)|

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses. 

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Vertex AI SDK and other required packages


In [3]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [4]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [4]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [6]:
PROJECT_ID = "qwiklabs-gcp-01-fbbf4f99eed0"  # @param {type:"string"}
LOCATION = "us-west1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [7]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import time

### Load model

In [8]:
model = GenerativeModel("gemini-1.5-flash")

In [9]:
import time

def call_gemini(prompt, generation_config=GenerationConfig(temperature=1.0)):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, generation_config=generation_config).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

def send_message_gemini(model, prompt):    
    wait_time = 1
    while True:
        try:
            response = model.send_message(prompt).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [10]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

print(call_gemini(prompt))

Here are some names for a flower shop that specializes in dried flowers, playing on various themes:

**Evoking Timelessness and Durability:**

* Everlasting Blooms
* Dried & Delightful
* The Timeless Bouquet
* The Everlasting Garden
* Bloom & Preserve
* Dried Flower Haven

**Emphasizing Uniqueness & Character:**

* The Dried Flower Studio
* The Boho Bouquet
* The Wildflower Workshop
* The Rustic Rose
* Bloom & Gather
* The Dried Flower Boutique

**Focusing on the Process:**

* The Pressed Flower Gallery
* The Dried Flower Collective
* The Flower Preservation Studio
* The Dried & Designed
* From Fresh to Forever

**Playful & Engaging Names:**

* The Flower Whisperer
* Dried & Daring
* The Bloom Room
* Flowerful Forever
* Petal & Preserve
* The Dried Flower Apothecary

**For a more modern and minimalist feel:**

* Bloom & Co.
* The Flower Lab
* Petal & Stone
* Dried.
* The Bouquet Bar

**Tips for choosing a name:**

* **Consider your target audience.** What kind of name will appeal to th

✅ Recommended. The prompt below is to the point and concise.

In [11]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(call_gemini(prompt))

## Flower Shop Names with a Dried Flower Vibe:

**Short & Sweet:**

* Bloom & Wither
* Dried Blooms
* Whispers & Blooms
* Petal & Sprig
* Everlasting Blooms
* The Dried Bouquet
* Dusty Rose
* Bloom & Co.
* Dried & Delicate
* Petal Preservation

**Descriptive & Evocative:**

* The Timeless Bouquet
* Forever in Bloom
* The Dried Flower Studio
* Botanical Whispers
* The Dried Bloom Apothecary
* The Rustic Bouquet
* The Vintage Flower Shop
* The Ethereal Bloom
* The Dried Flower Garden
* The Sun-Kissed Bloom

**Playful & Unique:**

* The Flower Faders
* The Bloom & Bustle
* The Dried & Dandy
* Whispering Petals
* The Dusty Bloom Collective
* The Paper Flower Garden
* The Petal Preservation Society
* The Bloom & Beyond
* The Eternal Bloom
* The Dried Flower Fairy

**Bonus Tip:**

* Consider your target audience and the overall style of your shop.
* Check for available domain names and social media handles.
* Get feedback from friends and family on your favorite choices. 

Good luck finding 

### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [12]:
prompt = "Tell me about Earth"

print(call_gemini(prompt))

## Earth: Our Home Planet

Earth is the third planet from the Sun and the only known celestial body to harbor life. It's a dynamic and complex world with a rich history, diverse ecosystems, and fascinating geological features. Let's explore some key facts about our planet:

**Physical Characteristics:**

* **Diameter:** 12,742 kilometers (7,926 miles)
* **Mass:** 5.972 × 10^24 kilograms
* **Surface area:** 510.1 million square kilometers (196.9 million square miles)
* **Composition:** Primarily iron, nickel, silicon, oxygen, magnesium, sulfur, and aluminum
* **Atmosphere:** Composed mainly of nitrogen (78%) and oxygen (21%)

**Geological Features:**

* **Continents:** Seven major landmasses (Asia, Africa, North America, South America, Antarctica, Europe, Australia)
* **Oceans:** Cover about 71% of the Earth's surface, with the Pacific Ocean being the largest.
* **Mountains:**  Himalayas, Andes, Rocky Mountains, and many more.
* **Volcanoes:** Active and extinct volcanoes, some of which

✅ Recommended. The prompt below is specific and well-defined.

In [13]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(call_gemini(prompt))

## What makes Earth unique compared to other planets:

**1. Abundant Liquid Water:**
* Earth is the only known planet with vast amounts of liquid water on its surface, essential for life as we know it.
* This liquid water exists in oceans, lakes, rivers, and even underground, creating a dynamic hydrological cycle.

**2. Plate Tectonics:**
* Earth's crust is divided into plates that constantly shift and interact, causing earthquakes, volcanoes, and mountain formation.
* This geological activity recycles materials and regulates climate over long periods.

**3. Atmospheric Composition:**
* Earth's atmosphere is rich in oxygen, a byproduct of life, which is vital for respiration.
* The atmosphere also protects us from harmful solar radiation and moderates temperatures.

**4. Diverse Life Forms:**
* Earth is teeming with an incredible diversity of life, from microscopic bacteria to giant whales.
* This biodiversity is a result of billions of years of evolution and adaptation, and its comple

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [14]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(call_gemini(prompt))

Here's a breakdown of the best method for boiling water and the reason behind the sky's blue hue:

**Boiling Water**

**Best Method:**

1. **Use a kettle or pot with a wide base:** This allows for quicker and more even heat distribution.
2. **Fill the container with the desired amount of water:**  Don't overfill, leaving room for the water to expand as it boils.
3. **Place the container on a high heat source:** This will bring the water to a boil faster.
4. **Watch for boiling:** The water will start to bubble vigorously when it reaches boiling point (212°F or 100°C).
5. **Turn off the heat source:** Once the water is boiling, remove it from the heat source.

**Why this is the best method:**

* **Efficiency:**  A wider base and high heat allow for faster heating. 
* **Safety:**  Leaving room for expansion prevents spills and potential burns.
* **Control:**  Turning off the heat once boiling is achieved prevents the water from over-boiling and potentially evaporating completely.

**The 

✅ Recommended. The prompts below asks one task a time.

In [15]:
prompt = "What's the best method of boiling water?"

print(call_gemini(prompt))

There isn't one single "best" method for boiling water, as the best method depends on your needs and situation. Here's a breakdown of common methods and when they might be best:

**Methods:**

* **Stovetop:**
    * **Pros:**  Versatile, can boil large quantities, readily available.
    * **Cons:** Can take longer than other methods, requires close supervision.
    * **Best for:**  Boiling large quantities of water, using a pot with a specific size or shape.

* **Electric Kettle:**
    * **Pros:**  Fast, convenient, automatic shut-off for safety, energy-efficient.
    * **Cons:**  Limited capacity, can be noisy, may not be suitable for all types of water.
    * **Best for:**  Quick boiling of small amounts of water (for tea, instant coffee, etc.), energy-conscious users.

* **Microwave:**
    * **Pros:**  Extremely fast, convenient, uses less energy than stovetop.
    * **Cons:**  Limited capacity, can create uneven heating, not suitable for all types of containers.
    * **Best for:** 

In [16]:
prompt = "Why is the sky blue?"

print(call_gemini(prompt))

The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's how it works:

* **Sunlight is a mixture of all colors:**  The sun emits light that contains all the colors of the rainbow.
* **Air molecules scatter light:** When sunlight enters the Earth's atmosphere, it encounters tiny particles like nitrogen and oxygen molecules. These molecules are much smaller than the wavelengths of visible light.
* **Shorter wavelengths scatter more:**  Blue light has a shorter wavelength than other colors in the visible spectrum.  When sunlight hits these tiny air molecules, the shorter wavelengths (blue and violet) are scattered more strongly than longer wavelengths (red and orange).
* **We see scattered blue light:** This scattered blue light is what we see as the blue sky.

**Why is the sky not violet?**

While violet light is scattered even more than blue, our eyes are more sensitive to blue light. So, we perceive the sky as blue, even though violet light is scattered more str

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to 1.0 so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer.

In [17]:
generation_config = GenerationConfig(temperature=1.0)

prompt = "What day is it today?"

print(call_gemini(prompt, generation_config))

I do not have access to real-time information, including the current date. To find out what day it is today, I recommend checking a calendar or your device's clock. 



Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini#system-instructions).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [18]:
model_travel = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=[
        "Hello! You are an AI chatbot for a travel web site.",
        "Your mission is to provide helpful queries for travelers.",
        "Remember that before you answer a question, you must check to see if it complies with your mission.",
        "If not, you can say, Sorry I can't answer that question.",
    ],
)

chat = model_travel.start_chat()

prompt = "What is the best place for sightseeing in Milan, Italy?"

print(send_message_gemini(chat, prompt))

Milan offers a wealth of sightseeing options! To give you the best recommendation, I need a little more information. What kind of sights are you interested in? 

For example, are you looking for:

* **Iconic landmarks:** Like the Duomo, Galleria Vittorio Emanuele II, or Castello Sforzesco?
* **Art and culture:** Perhaps you'd like to visit the Pinacoteca di Brera, the Museo del Novecento, or the Teatro alla Scala?
* **Fashion and design:** Maybe you're interested in the Quadrilatero della Moda, the Museo del Design, or the Triennale di Milano?

Tell me what you're most interested in seeing, and I can point you to the best places! 



Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [19]:
prompt = "What's for dinner?"

print(send_message_gemini(chat, prompt))

Sorry, I can't answer that question. I'm designed to help travelers plan their trips, and I don't have access to real-time information like restaurant menus. 

However, I can help you find great places to eat in Milan! What kind of cuisine are you interested in?  Tell me your preferences and I can suggest some delicious restaurants. 



You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [20]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(call_gemini(prompt))

## Programming Activities for High School Students:

Here are some programming activities tailored to high school students, aiming to improve your skills in a fun and engaging way:

**Beginner Level:**

* **Build a simple calculator:**  This project teaches basic input/output, arithmetic operations, and conditional statements.
* **Create a text-based adventure game:**  This involves using loops, conditions, and basic game logic to create a simple story-driven game.
* **Develop a guessing game:**  This project focuses on user input, random number generation, and comparison operators.
* **Design a website with HTML, CSS, and JavaScript:**  Learn basic web development and create your own website with simple features.
* **Try Codecademy or Khan Academy courses:**  These online platforms offer interactive lessons and tutorials for beginners in various programming languages.

**Intermediate Level:**

* **Create a to-do list application:**  This project requires using data structures (lists o

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [21]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

print(call_gemini(prompt))

That's a great question! All three programming languages are valuable, but the best one for you depends on what you're interested in and what you want to achieve. Here's a breakdown to help you decide:

**a) Learn Python:**

* **Pros:** 
    * **Easy to learn:** Python is known for its beginner-friendly syntax, making it a great starting point for anyone new to programming. 
    * **Versatile:** Python is used in a wide range of fields, including web development, data science, machine learning, and scientific computing.
    * **Large and active community:** There's a huge community of Python developers, meaning you'll find plenty of resources, support, and libraries to help you along the way.
* **Cons:**
    * **Not as fast as some other languages:** Python is interpreted, which means it's slightly slower than compiled languages like C++ or Fortran.
    * **Not the best for performance-critical applications:**  While Python is used in some high-performance areas, other languages might 

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [22]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 



#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [23]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **negative** 



#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [24]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **positive** 

Here's why:

* **"Surprised me about this video - it was actually original"**: This implies the speaker was pleasantly surprised, suggesting a positive experience.
* **"Not the same old recycled stuff"**:  This indicates the speaker finds the video refreshing and different, a positive attribute.
* **"Watch it - you will not regret it"**: This is a strong recommendation, further reinforcing the positive sentiment. 



#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.